<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Liczenie_IOU_list/colabs/Wczytanie_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup sparka

# Załadowanie nowych baz

##Setup sparka

In [ ]:
!pip install pyspark py4j
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 200 kB 57.7 MB/s 
     |████████████████████████████████| 199 kB 12.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=e77396734310b68f8d253bdd86d7badadaa34af007a635cf5528a1f39347a13e
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
--2022-12-07 23:21:22--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274099817 (261M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop2.tgz’

spark-3.3.1-bin-had 100%[===============

In [ ]:
import pyspark
import findspark
from pyspark.sql import SparkSession
import os
import pandas as pd
from pyspark.sql.functions import split, explode, collect_list, first, array_distinct, min, max
from google.colab import drive

In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
spark=SparkSession.builder.appName('Colab').getOrCreate()
spark

#Pobranie danych

##Import danych

In [ ]:
!wget https://datasets.imdbws.com/name.basics.tsv.gz
!wget https://datasets.imdbws.com/title.akas.tsv.gz
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!wget https://datasets.imdbws.com/title.crew.tsv.gz
!wget https://datasets.imdbws.com/title.episode.tsv.gz
!wget https://datasets.imdbws.com/title.principals.tsv.gz
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-12-07 23:21:42--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.224.2.116, 13.224.2.111, 13.224.2.40, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.224.2.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 237410314 (226M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

name.basics.tsv.gz  100%[===================>] 226.41M  25.8MB/s    in 7.5s    

2022-12-07 23:21:50 (30.3 MB/s) - ‘name.basics.tsv.gz’ saved [237410314/237410314]

--2022-12-07 23:21:50--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.224.2.116, 13.224.2.111, 13.224.2.40, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.224.2.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 289723255 (276M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 276.30M  43.9MB/

##Rozpakowanie danych

In [ ]:
!gzip -dc /content/name.basics.tsv.gz > name.basics.csv
!gzip -dc /content/title.akas.tsv.gz > title.akas.csv
!gzip -dc /content/title.basics.tsv.gz > title.basic.csv
!gzip -dc /content/title.crew.tsv.gz > title.crew.csv
!gzip -dc /content/title.episode.tsv.gz > title.episode.csv
!gzip -dc /content/title.principals.tsv.gz > title.principals.csv
!gzip -dc /content/title.ratings.tsv.gz > title.ratings.csv

#Wczytanie danych

##Wczytajmy dane z rozpakowanych plików

In [ ]:
df_name_basics=spark.read.option("header", "true").option("delimiter", "\t").csv('name.basics.csv' ) 
df_title_akas=spark.read.option("header","true").option("delimiter", "\t").csv('title.akas.csv')
df_title_basic=spark.read.option("header","true").option("delimiter", "\t").csv('title.basic.csv')
df_title_crew=spark.read.option("header","true").option("delimiter", "\t").csv('title.crew.csv')
df_title_episode=spark.read.option("header","true").option("delimiter","\t").csv('title.episode.csv')
df_title_principals=spark.read.option("header","true").option("delimiter","\t").csv('title.principals.csv')
df_title_ratings=spark.read.option("header","true").option("delimiter","\t").csv('title.ratings.csv')

##Wyświetlmy dane

In [ ]:
df_name_basics.show(3)
df_title_akas.show(3)
df_title_basic.show(3)
df_title_crew.show(3)
df_title_episode.show(3)
df_title_principals.show(3)
df_title_ratings.show(3)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0031983,tt00723...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0038355,tt01170...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0049189,tt00573...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 3 rows

+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Карменсіта|    

# Złączenie istotnych kolumn

## Wybierzmy z tabel tylko istotne rekordy:
- df_title_basic: wybierzmy tylko rekordy z primaryProfession zawierającym "actor" lub "actress",
- df_title_principals: wybierzmy rekordy z category zawierającym "actor" lub "actress".

In [ ]:
df_name_basics_selected = df_name_basics.filter("primaryProfession like '%actor%' or primaryProfession like '%actress%'")
df_title_principals_selected = df_title_principals.filter((df_title_principals.category == "actor") | (df_title_principals.category == "actress"))

## Wybierzmy następujące kolumny z tabel:
- df_title_basic:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - titleType (rodzaj tytułu, np. film, serial - można porównywać podobieństwo aktorów na podstawie tego w jakich typach produkcji grali)
  - originalTitle (tytuł produkcji - być może przyda się do analizy podobnych filmów)
  - isAdult (czy produkcja jest dla dorosłych - można wykorzystać do porównywania aktorów na podstawie tego czy grają głównie w produkcjach dla dorosłych)
  - startYear (data wypuszczenia produkcji - można wykorzystać do porównania epok filmów, w których grali aktorzy)
  - endYear (data zakończenia serii, w innych przypadkach \N - można wykorzystać do porównania epok filmów, w których grali aktorzy)
  - genres (gatunki - może posłużyć do porównania aktorów na podstawie tego, że grali w produkcjach o podobnych gatunkach)

- df_title_principals:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - nconst (unikalny numer aktora - potrzebne do złączenia tabel)
  - category (kategoria pracy (actor/actress) - może posłużyć do porównania aktorów na podstawie takiej samej płci)
  - characters (postacie zagrane w produkcji - może posłużyć do porównania aktorów na podstawie podobnych granych postaci)

- df_name_basics:
  - nconst(unikalny numer aktora - potrzebne do złączenia tabel)
  - primaryName (imię i nazwisko aktora - potrzebne do wyświetlania aktora lub do odnajdywania go w bazie po imieniu i nazwisku)
  - knownForTitles (tytuły produkcji, z których znany jest aktor - może posłużyć do porównania aktorów na podstawie tych samych lub podobnych produkcji, z których są znani)

In [ ]:
df_title_basic_selected = df_title_basic.select(["tconst", "titleType", "originalTitle", "isAdult", "startYear", "endYear", "genres"])
df_title_principals_selected = df_title_principals_selected.select(["tconst", "nconst", "category", "characters"])
df_name_basics_selected = df_name_basics_selected.select(["nconst", "primaryName", "knownForTitles"])

# Złączmy tabele df_name_basics, df_title_principals i df_title_basic

In [ ]:
print("df_name_basics_selected dataframe size: ", (df_name_basics_selected.count(), len(df_name_basics_selected.columns)))
print("df_title_principals_selected dataframe size: ", (df_title_principals_selected.count(), len(df_title_principals_selected.columns)))
print("df_title_basic_selected dataframe size: ", (df_title_basic_selected.count(), len(df_title_basic_selected.columns)))
data = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(df_name_basics_selected, "nconst", "inner")
print("joined dataframe size: ", (data.count(), len(data.columns)))

df_name_basics_selected dataframe size:  (4438669, 3)
df_title_principals_selected dataframe size:  (20916123, 4)
df_title_basic_selected dataframe size:  (9427157, 7)
joined dataframe size:  (20916123, 10)
joined dataframe size:  (20292236, 12)


## Łączymy dane na temat jednego aktora w jeden rekord

In [ ]:
data = data.groupby('nconst').agg(collect_list('tconst').alias("tconst"), collect_list('titleType').alias("titleType"), collect_list('originalTitle').alias("originalTitle"), collect_list('isAdult').alias("isAdult"), min('startYear').alias("startYear"), max('endYear').alias("endYear"), collect_list('genres').alias("genres"), first('category').alias("category"), collect_list('characters').alias("characters"), first('primaryName').alias("primaryName"), first('knownForTitles').alias("knownForTitles"))


# Wybierzmy przykładowe wiersze z końcowego dataframe'u

In [ ]:
sample = data.rdd.takeSample(False, 5)

In [ ]:
sample = pd.DataFrame(sample, columns = data.columns)
sample

,nconst,tconst,titleType,originalTitle,isAdult,startYear,endYear,genres,category,characters,primaryName,knownForTitles
0,nm0842199,"[tt0348872, tt0085613, tt4067520, tt0181243, t...","[tvMovie, short, movie, tvEpisode, tvSeries, s...","[Skulduggery, Goodie-Two-Shoes, Schadenfreude,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1982,\N,"[Drama, Romance,Short, Comedy,Crime,Mystery, C...",actor,"[[""Sweeney""], [""Spicer""], [""Little Piss Head""]...",Steve Sweeney,"tt0330468,tt0119792,tt0120735,tt0159191"
1,nm8616733,[tt8661400],[video],[La gabbia],[0],2017,\N,"[Drama,Short]",actress,"[[""Charlotte""]]",Clarissa Rollo,"tt6954640,tt8661400,tt6289828"
2,nm13773316,[tt8793478],[movie],[Yawahada no okite],[0],1966,\N,[\N],actress,[\N],Kenji Hasegawa,tt8793478
3,nm1157474,[tt0449912],[tvSpecial],[The Doobie Brothers: Rockin' Down the Highway...,[0],1996,\N,[Music],actor,"[[""Performer""]]",Tom Johnston,"tt0165854,tt0245686,tt0092890,tt1596345"
4,nm11240422,[tt12637020],[short],[Under Fire - a Star Trek fan production],[0],2020,\N,"[Sci-Fi,Short]",actress,"[[""Chief Surgeon Kavanaugh""]]",LaCindra McGowan,tt11534748


# Załadowanie nowych baz

   **W celu załadowania nowych danych należy:**


1. Pobrać token API:
  * Zalogować się na stronę *www.kaggle.com*
  * Kliknąć "Account"
  * Znaleźć zakładkę API i nacisnąć przycisk "*Create New API Token*" - w tym miejscu pobierany jest plik *kaggle.json* - API
2. Utworzyć plik na swoim dysku pod nazwą "Big Data"
3. Wrzucić pobrane API (plik *kaggle.json*) na swój dysk
4. Uruchomić kod! :)




## Wczytajmy bazę oskarów: "Best Movies Watchlist"


In [ ]:
drive.mount('/content/gdrive')
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Big Data"

Mounted at /content/gdrive


In [ ]:
! pip install kaggle
! mkdir ~/.kaggle 
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
! kaggle datasets download unanimad/the-oscar-award #załadujmy dane
! unzip the-oscar-award.zip #rozpakujmy z zip'a
oscars=spark.read.option("header","true").csv('the_oscar_award.csv')
oscars.show(5)


  0% 0.00/191k [00:00<?, ?B/s]
100% 191k/191k [00:00<00:00, 11.4MB/s]
Archive:  the-oscar-award.zip
  inflating: the_oscar_award.csv     
+---------+-------------+--------+--------+-------------------+----------------+------+
|year_film|year_ceremony|ceremony|category|               name|            film|winner|
+---------+-------------+--------+--------+-------------------+----------------+------+
|     1927|         1928|       1|   ACTOR|Richard Barthelmess|       The Noose| False|
|     1927|         1928|       1|   ACTOR|      Emil Jannings|The Last Command|  True|
|     1927|         1928|       1| ACTRESS|     Louise Dresser| A Ship Comes In| False|
|     1927|         1928|       1| ACTRESS|       Janet Gaynor|      7th Heaven|  True|
|     1927|         1928|       1| ACTRESS|     Gloria Swanson|  Sadie Thompson| False|
+---------+-------------+--------+--------+-------------------+----------------+------+
only showing top 5 rows



Main Data:
* primaryName	- aktor
* originalTitle - nazwa filmu

W nowej bazie oskarów:
* name - aktor
* film - film

## Wczytajmy bazę złotych globów: "Golden Globe Awards, 1944 - 2020"

In [ ]:
! kaggle datasets download unanimad/golden-globe-awards
! unzip golden-globe-awards.zip
globe=spark.read.option("header","true").csv('golden_globe_awards.csv')
globe.show(5)

  0% 0.00/117k [00:00<?, ?B/s]
100% 117k/117k [00:00<00:00, 41.7MB/s]
Archive:  golden-globe-awards.zip
  inflating: golden_globe_awards.csv  
+---------+----------+--------+--------------------+--------------------+--------------------+----+
|year_film|year_award|ceremony|            category|             nominee|                film| win|
+---------+----------+--------+--------------------+--------------------+--------------------+----+
|     1943|      1944|       1|Best Performance ...|      Katina Paxinou|For Whom The Bell...|True|
|     1943|      1944|       1|Best Performance ...|       Akim Tamiroff|For Whom The Bell...|True|
|     1943|      1944|       1|Best Director - M...|          Henry King|The Song Of Berna...|True|
|     1943|      1944|       1|             Picture|The Song Of Berna...|                null|True|
|     1943|      1944|       1|Actress In A Lead...|      Jennifer Jones|The Song Of Berna...|True|
+---------+----------+--------+--------------------+-----

## Wczytajmy bazę złotych globów: "Primetime Emmy Awards, 1949-2017"

In [ ]:
! kaggle datasets download unanimad/emmy-awards
! unzip emmy-awards.zip 
emmy_awards_category=spark.read.option("header","true").csv('emmy_awards_categories.csv')
emmy_awards_category.show(5)


  0% 0.00/904k [00:00<?, ?B/s]
100% 904k/904k [00:00<00:00, 45.2MB/s]
Archive:  emmy-awards.zip
  inflating: emmy_awards_categories.csv  
  inflating: the_emmy_awards.csv     
+------------------+--------+----+
|              name|category|role|
+------------------+--------+----+
|     Comedy Series|  Comedy|main|
|        Lead Actor|  Comedy|main|
|      Lead Actress|  Comedy|main|
|  Supporting Actor|  Comedy|main|
|Supporting Actress|  Comedy|main|
+------------------+--------+----+
only showing top 5 rows



In [ ]:
emmy_awards=spark.read.option("header","true").csv('the_emmy_awards.csv')
emmy_awards.show(5)

+---+----+--------------------+--------------------+--------------------+--------+--------------------+-----+
| id|year|            category|             nominee|               staff| company|            producer|  win|
+---+----+--------------------+--------------------+--------------------+--------+--------------------+-----+
|  1|2019|Outstanding Chara...|        The Simpsons|Hank Azaria, as M...|     FOX|Gracie Films in a...|False|
|  2|2019|Outstanding Chara...|          Family Guy|Alex Borstein, as...|     FOX|20th Century Fox ...|False|
|  3|2019|Outstanding Chara...|When You Wish Upo...|Eric Jacobson, as...|     HBO|Sesame Street Wor...|False|
|  4|2019|Outstanding Chara...|     F Is For Family|Kevin Michael Ric...| Netflix|Wild West Televis...|False|
|  5|2019|Outstanding Produ...| Escape At Dannemora|Mark Ricker, Prod...|Showtime|Red Hour, Busyhan...|False|
+---+----+--------------------+--------------------+--------------------+--------+--------------------+-----+
only showi

## Zaczytanie danych TMDB 

   informacje o aktorach i osobach pracujących przy produkcji filmów oraz budżetach, gatunkach, popularności, wytwórniach filmowych, oryginalnyh językach i tytułach


In [ ]:
! kaggle datasets download tmdb/tmdb-movie-metadata
! unzip tmdb-movie-metadata.zip
tmdb_credits=spark.read.option("header","true").csv('tmdb_5000_credits.csv')
tmdb_movies=spark.read.option("header","true").csv('tmdb_5000_movies.csv')

 56% 5.00M/8.89M [00:00<00:00, 29.8MB/s]
100% 8.89M/8.89M [00:00<00:00, 46.4MB/s]
Archive:  tmdb-movie-metadata.zip
  inflating: tmdb_5000_credits.csv   
  inflating: tmdb_5000_movies.csv    


In [ ]:
tmdb_credits.show(5)

+--------+--------------------+-------------------+--------------------+
|movie_id|               title|               cast|                crew|
+--------+--------------------+-------------------+--------------------+
|   19995|              Avatar|"[{""cast_id"": 242| ""character"": "...|
|     285|Pirates of the Ca...|  "[{""cast_id"": 4| ""character"": "...|
|  206647|             Spectre|  "[{""cast_id"": 1| ""character"": "...|
|   49026|The Dark Knight R...|  "[{""cast_id"": 2| ""character"": "...|
|   49529|         John Carter|  "[{""cast_id"": 5| ""character"": "...|
+--------+--------------------+-------------------+--------------------+
only showing top 5 rows



In [ ]:
tmdb_movies.show(5)

+---------+-------------+--------------------+------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+--------------+--------------------+--------------+--------------------+---------------+--------------------+----------------+
|   budget|       genres|            homepage|          id|            keywords|original_language|      original_title|            overview|          popularity|production_companies|production_countries|   release_date|             revenue|       runtime|    spoken_languages|        status|             tagline|          title|        vote_average|      vote_count|
+---------+-------------+--------------------+------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+--------------+-----------

# Metody używane do liczenia similarity

In [1]:
def IOU(lista1, lista2):
  # metoda przyjmuje jako argument dwie listy i zwraca ich indeks Jaccarda, czyli liczbę z przedzialu [0,1]
  intersection = []
  union = []
  for element in lista1:
    union.append(element)
    if element in lista2:
      intersection.append(element)
  for element in lista2:
    if element not in union:
      union.append(element)
  IOU = len(intersection)/len(union)
  return IOU